### This code is to demo a Document Agent
1. Will read three PDF documents and create a vectorstore + retreiver objects for each
2. Create three Tools using the retreiver objects to extract information from the vectorstores
3. Create agents with the three tools
4. Invoke the Agent with a query, the agent should decide on which Tool to execute based on the query

In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.agent.openai import OpenAIAgent 
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, ListIndex, SimpleDirectoryReader
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from dotenv import load_dotenv
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import get_response_synthesizer
import os

In [2]:
# Load the OpenAI API Key into the environment variable named OPENAI_API_KEY
load_dotenv()   
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [3]:
# Set up model configuration
Settings.llm = OpenAI(temperature=0.5,
    model="gpt-3.5-turbo") 

In [4]:
# Load 3 PDF documents on mental Haalth from the folder, using SimpleDirectoryReader

titles = [
    "EJ1154566", 
    "sg-youth-mental-health-social-media-advisory", 
    "what_is_mental_illness"
    ]

documents = {}
for title in titles:
    documents[title] = SimpleDirectoryReader(input_files=[f"data/{title}.pdf"]).load_data()
print(f"loaded documents with {len(documents)} documents")

loaded documents with 3 documents


In [5]:
documents["sg-youth-mental-health-social-media-advisory"]

[Document(id_='78b2db0f-aad9-45e8-8c9f-62a490670d62', embedding=None, metadata={'page_label': '1', 'file_name': 'sg-youth-mental-health-social-media-advisory.pdf', 'file_path': 'data/sg-youth-mental-health-social-media-advisory.pdf', 'file_type': 'application/pdf', 'file_size': 1029053, 'creation_date': '2024-05-22', 'last_modified_date': '2023-06-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Social Media  \nand Y outh  Mental Health\nThe U.S. Surgeon General’s Advisory\n2023\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='a8bd0681-bcb3-4fac-a763-1409f18a04b6', embedding=None, metadata={'page_label': '2', 'file_name': 'sg-youth-me

In [6]:
# Iteratively, build three Tools - one for each document, with theirr corresponding retreiver objects

query_engine_tools = []
response_synthesizer = get_response_synthesizer(response_mode=ResponseMode.COMPACT)
for title in titles:

    # build vector index
    vector_index = VectorStoreIndex.from_documents(documents[title])
    retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=3)
    # define query engines
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7, # filter nodes with similarity score below the cutoff 
                                                    filter_empty=True,  # filter empty nodes
                                                    filter_duplicates=True,  # filter duplicate nodes
                                                    filter_similar=True,  # filter similar nodes
                                                    )],
        response_synthesizer=response_synthesizer,                                                 
    )

    # define tools
    query_engine_tools += [
        QueryEngineTool(
            query_engine=query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{title}",
                description=f"Useful for retrieving specific context related to {title}"
            ),
        )
        ] 

# build agent
agent = OpenAIAgent.from_tools(
    query_engine_tools,
    verbose=True,
)

In [7]:
# print the agent and the number, names of tools
print("Agent: ", agent)
print("Number of Tools: ", len(query_engine_tools))
print("Tool Names: ", [tool.metadata.name for tool in query_engine_tools])

Agent:  <llama_index.agent.openai.base.OpenAIAgent object at 0x7fafecfec0a0>
Number of Tools:  3
Tool Names:  ['vector_tool_EJ1154566', 'vector_tool_sg-youth-mental-health-social-media-advisory', 'vector_tool_what_is_mental_illness']


In [8]:
agent.query("What is mental health?")

Added user message to memory: What is mental health?
=== Calling Function ===
Calling function: vector_tool_what_is_mental_illness with args: {"input":"What is mental health?"}
Got output: Mental health refers to a person's overall emotional, psychological, and social well-being. It involves how individuals think, feel, and act, and encompasses their ability to handle stress, relate to others, and make choices. Good mental health contributes to one's overall quality of life and helps individuals cope with the challenges they face.



Response(response="Mental health refers to a person's overall emotional, psychological, and social well-being. It involves how individuals think, feel, and act, and encompasses their ability to handle stress, relate to others, and make choices. Good mental health contributes to one's overall quality of life and helps individuals cope with the challenges they face.", source_nodes=[NodeWithScore(node=TextNode(id_='096bfc91-9a5c-45ef-ab69-ec4c74c955b8', embedding=None, metadata={'page_label': '2', 'file_name': 'what_is_mental_illness.pdf', 'file_path': 'data/what_is_mental_illness.pdf', 'file_type': 'application/pdf', 'file_size': 228227, 'creation_date': '2024-05-05', 'last_modified_date': '2020-01-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelation

In [9]:
agent.query("What are the Potential Risk of Harm from Content Exposure?").response

Added user message to memory: What are the Potential Risk of Harm from Content Exposure?
=== Calling Function ===
Calling function: vector_tool_EJ1154566 with args: {"input": "Potential Risk of Harm from Content Exposure"}
Got output: The potential risk of harm from content exposure should be carefully considered, especially when dealing with sensitive topics related to mental health. It is essential to be mindful of how exposure to certain content can impact individuals' well-being and mental health. Establishing appropriate guidelines and providing support resources can help mitigate any potential risks associated with content exposure.

=== Calling Function ===
Calling function: vector_tool_sg-youth-mental-health-social-media-advisory with args: {"input": "Potential Risk of Harm from Content Exposure"}
Got output: The potential risk of harm from content exposure includes the accessibility of extreme, inappropriate, and harmful content to children and adolescents through social media

"The potential risks of harm from content exposure include:\n\n1. The impact on individuals' well-being and mental health, especially when dealing with sensitive topics related to mental health. It is crucial to consider how exposure to certain content can affect mental health and establish appropriate guidelines and support resources to mitigate risks.\n\n2. Accessibility of extreme, inappropriate, and harmful content to children and adolescents through social media platforms. This content can include self-harm acts, risky challenges, and harmful behaviors that may normalize such actions and lead to negative mental health outcomes, especially for those already experiencing mental health difficulties.\n\n3. Social media's role in perpetuating body dissatisfaction, disordered eating behaviors, social comparison, and low self-esteem, particularly among adolescent girls.\n\n4. Exposure to hate-based content, with a significant number of adolescents reporting exposure to racist content or 